In [6]:
import numpy as np
import pandas as pd
import tokenize
import naming as naming
import suji as sj
from crawling import getText, getOneText
import io
from time import time
import ast

class Feature:
    __slots__ = ['name', 'users']
    def __init__(self, name):
        self.name = name
        
        users_link = 'http://114.71.103.80/new_db.php?q=select+*+from+users%0D%0Alimit+200%0D%0A%0D%0A'
        self.users = getText(users_link, '1')

    def make_df(feature_func):
        def code_link(user):
            before = 'http://114.71.103.80/new_db.php?q=select+*+from+code_submission+c%2C+%28select+id%2C+max%28regtime%29+a+from+code_submission+where+id+%3D'
            after = '+group+by+p_no%29+d+where+c.id+%3D+d.id+and+d.a+%3D+c.regtime+'
            return before + user + after
        pass 
    
    def cleaning(self):
        raise NotImplementedError()
    
class NumberFeature(Feature):
    pass

class ListFeature(Feature):
    pass

f = Feature('hi')
f.problems

In [17]:

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

users_link = 'http://114.71.103.80/new_db.php?q=select+*+from+users%0D%0Alimit+200%0D%0A%0D%0A'
def code_link(user):
    before = 'http://114.71.103.80/new_db.php?q=select+*+from+code_submission+c%2C+%28select+id%2C+max%28regtime%29+a+from+code_submission+where+id+%3D'
    after = '+group+by+p_no%29+d+where+c.id+%3D+d.id+and+d.a+%3D+c.regtime+'
    return before + user + after


users = getText(users_link, '1')
total_num_p = 78
     

def make_feature_df(feature_func, users, total_num_p = total_num_p):
    df = pd.DataFrame(columns=users,index=range(1,total_num_p + 1))
    start = time()
    for user in users:
        num_p = getText(code_link(user),  '2')
        num_p = list(map(int, num_p))
        problems = getText(code_link(user),  '4')
        results=[]
        for problem in problems:
            stream = io.StringIO(problem)
            tokens = tokenize.generate_tokens(stream.readline)
            try:
                results.append(feature_func(tokens))
            except:
                results.append(None) # None = error in code

        result_i = 0
        for i, v in enumerate(num_p):
            df.loc[v][user] = results[i]
        # NaN = problem isn't submitted
    end = time()
    print(f'Running time {end - start} sec')
    return df

def avg_cleaning(df): #suji
    df.loc[:,'mean'] = df.mean(axis=1)
    df = df.divide(df['mean'], axis = 0)
    df = df.drop(['mean'], axis=1)
    return df

def self_cleaning(df): # yeram
    num_p, num_u = df.shape
    for i in range(1, num_p + 1):
        for j in range(num_u):
            lst=df.loc[i][j]
            if type(lst) != list:
                pass
            else:
                last_num=len(lst)-1
                for k in range(len(lst)):
                    if lst[last_num] != 0:
                        lst[k]=lst[k]/lst[last_num]
                del lst[last_num]
                df.loc[i][j]=lst
    return df


def integrate(dfs, prob, user): #seulin
    def check_shape(dfs): # check the shape of all dfs
        shape = dfs[0].shape
        for df in dfs:
            assert(shape == df.shape)
    check_shape(dfs)
    res = []
    for df in dfs:
        feature = df.loc[prob][user]
        if type(feature) == list:
            res.extend(feature)
        elif type(feature) in [int, float]:
            res.append(feature)
        else:
            # NaN
            pass
    return res


# get_distribution을 위한 함수, df에서 일부 레벨 별로 df를 추출함
def get_problems_users(df, level, users):
    return df.loc[(level-1)*10+1:level*10, users]

def get_std_users(df, level, users):
    stds = []
    for u in users:
        stds.append(get_std(level, u))
    return pd.Series(stds, index=users)

def get_std(df, level, user):
    def get_problems(df, level, user):
        #level 1 : 1~10번 문제, 2: 11~20번 문제
        return df.loc[(level-1)*10+1:level*10][user]
    series = get_problems(df, level, user)
    if series.dtype in [int, float]:
        return series.std()
    elif series.dtype == object:
        lst = series.tolist()
        new = []
        for prob in lst:
            if type(prob) == list:
                new.append(prob[0]) #나머지 항들도 같은 std 값임.
        new = pd.Series(new)
        return new.std()
#         res = []
#         for i in range(len(lst[0])): # length of each list
#             new = pd.Series([prob[i] for prob in lst])
#             res.append(new.std())
#         return res
# 유지도 : 개인별로 10개의 문제 기록을 보고 표준편차를 계산, 표준편차가 작으면 높은 가중치

def calculate_plagiarism(origin, new, weights):
    assert origin.size == new.size == weights.size
    diff = origin - new
    diff = diff.abs()
    return diff*weights


In [4]:
def one_update(res, new_data, ratio = 0.7):
    if len(res) == 0:
#     ////////////////////////////////////////////////
#     왜 깊은 복사 안되지 df찍어보면 안다
        res  =  copy.deepcopy(new_data)
#     /////////////////////////////////////////////////
        return copy.deepcopy(new_data)
    
    data_size = len(res)
    arr_size = len(res[0])
    
    for i in range(data_size):
        for j in range(arr_size):
#             print(new_data[i])
            try:
                val = (1-ratio) * res[i][j] + ratio * new_data[i][j]
                res[i][j] = round(val, 5)
                
            except:
                if new_data[i] == None or np.isnan(new_data[i]):
                    print("nan, none 발생")
                    break
            
    return res
           
    
def total_update(data):
    data_size = len(df)
    res = []
    
    for i in range(1,data_size+1):
        res = one_update(res, data.loc[i])
        print("i=",i)
        print(res)
        
    return res


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

def get_distribution(data, n_clustering = 2):
    
    mean_df = data.mean()
    np_mean_Df = mean_df.to_numpy().reshape(-1, 1)
#     print(np_mean_Df)
    
    model = KMeans(n_clusters=n_clustering, init='k-means++', n_init=10, max_iter=300, tol=1e-4)
    model.fit(np_mean_Df)

    y_predict = model.labels_
    n_y_predict = len(y_predict)

    kmean_result = np.zeros(n_clustering) 
    
    for i in range(n_y_predict):
        kmean_result[y_predict[i]]+=1
    kmean_result /= n_y_predict
    
    result = [kmean_result[pred_y] for pred_y in y_predict]
    
    
    return pd.Series(result, index=list(data))




data = {
    '201811011': [90, 99, 50, 65],
    '20181167': [88, 76, 95, 79],
    '201811132' : [100, 99, 50, 65],
    '201811141': [0, 99, 40, 65]
}

# print(data)
df1 = pd.DataFrame(data)
print(df1)
get_distribution(df1)

   201811011  20181167  201811132  201811141
0         90        88        100          0
1         99        76         99         99
2         50        95         50         40
3         65        79         65         65


201811011    0.75
20181167     0.75
201811132    0.75
201811141    0.25
dtype: float64

In [11]:
# 정제한 df들 만들기

# for func in [sj. ~~~~] 함수들 자기 전에 돌리고 자자.
df1 = make_feature_df(sj.Annotation, users)
df1 = self_cleaning(df1)
df1.to_pickle("./dfs/Annotation.pkl")

df2 = make_feature_df(sj.OpenParOpBl, users)
df2 = self_cleaning(df2)
df2.to_pickle("./dfs/OpenParOpBl.pkl")

Running time 530.3547825813293 sec


In [20]:
# 저장한 df 불러오기
df1 = pd.read_pickle("./dfs/Annotation.pkl")
df2 = pd.read_pickle("./dfs/OpenParOpBl.pkl")

In [ ]:
#df2를 element df로 쪼개는 함수가 필요함.

In [ ]:
# feature 가중치 구하기
level = 1

# kmeans으로 학생들 사이의 분산 구하기
temp = get_problems_users(df1, 1, users)
print(get_distribution(temp))

# 표편으로 유지도 구하기
temp = get_std(df1. level, users)
print(temp)
####################################

# kmeans으로 학생들 사이의 분산 구하기
temp = get_problems_users(df2, 1, users)
print(get_distribution(temp))

# 표편으로 유지도 구하기
temp = get_std(df2. level, users)
print(temp)

In [5]:
# 새로 들어오는 문제의 코딩 스타일을 정의
integrate([df1, df2], 10, "201411023")

[0.0, 0.0, 0.0, 1.0, 1.153846153846154]

In [14]:
# save and read
df.to_pickle("./dfs/feature_df.pkl")
df_data = pd.read_pickle("./dfs/feature_df.pkl")